# [**Transformation de colonne avec des types mélangés**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_1_compose/plot_column_transformer_mixed_types.ipynb)<br/>([*Column Transformer with Mixed Types*](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html))

Cet exemple montre comment appliquer différentes pipelines de prétraitement et d'extraction de caractéristiques à différents sous-ensembles de caractéristiques en utilisant [**`ColumnTransformer`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html). Cela est particulièrement utile pour les jeux de données qui contiennent des types de données hétérogènes, car nous pouvons vouloir normaliser les caractéristiques numériques et one-hot encoder les caractéristiques catégorielles.

Dans cet exemple, les données numériques sont standardisées après une imputation de la moyenne. Les données catégorielles sont encodées via `OneHotEncoder`, qui crée une nouvelle catégorie pour les valeurs manquantes. Nous réduisons encore la dimensionnalité en sélectionnant les catégories en utilisant un test du chi-deux.

En outre, nous montrons deux façons différentes de distribuer les colonnes au pré-traitement particulier : par les noms de colonne et par les types de données de colonne.

Enfin, la pipeline de prétraitement est intégrée dans une pipeline de prédiction complète en utilisant [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), avec un modèle de classification simple.

In [1]:
# Author: Pedro Morales <part.morales@gmail.com>
#
# License: BSD 3 clause

In [2]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_selection import SelectPercentile, chi2

np.random.seed(0)

Chargement des données depuis https://www.openml.org/d/40945

In [3]:
X, y = fetch_openml(
    "titanic", version=1, as_frame=True, return_X_y=True, parser="pandas"
)

# Alternatively X and y can be obtained directly from the frame attribute:
# X = titanic.frame.drop('survived', axis=1)
# y = titanic.frame['survived']

## Utiliser `ColumnTransformer` en sélectionnant les colonnes par noms

Nous allons entraîner notre classificateur avec les caractéristiques suivantes :

Caractéristiques numériques :
* `age` : flottant ;
* `fare` : flottant.

Caractéristiques catégorielles :
* `embarked` : catégories encodées en chaînes de caractères `{'C', 'S', 'Q'}`;
* `sex` : catégories encodées en chaînes de caractères `{'female', 'male'}`;
* `pclass` : entiers ordinaux `{1, 2, 3}`.

Nous créons les pipelines de prétraitement pour les données numériques et catégorielles. Notez que `pclass` peut être tout autant traité comme une caractéristique catégorielle que numérique.

In [4]:
numeric_features = ["age", "fare"]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["embarked", "sex", "pclass"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

## Ajouter le classificateur au pipeline de prétraitement

Maintenant, nous avons un pipeline de prédiction complet.

In [5]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.798


## Représentation HTML du Pipeline (affichage du diagramme)

Lorsque le Pipeline est imprimé dans un carnet de jupyter, une représentation HTML de l'estimateur est affichée :

In [6]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=50,
                                                                                    score_func=<function chi2 at 0x000001C2AF6E6440>))]),
                                                  ['embarked', 'sex',
                                                   'pclass'])])),
                ('classifier', LogisticRegression())])

## Utiliser `ColumnTransformer` en sélectionnant les colonnes par types de données

Lors de la manipulation d'un jeu de données nettoyé, le prétraitement peut être automatique en utilisant les types de données de la colonne pour décider de traiter une colonne en tant que caractéristique numérique ou catégorielle. [**`sklearn.compose.make_column_selector`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_selector.html) offre cette possibilité. Tout d'abord, nous ne sélectionnons qu'un sous-ensemble de colonnes pour simplifier notre exemple.

In [7]:
subset_feature = ["embarked", "sex", "pclass", "age", "fare"]
X_train, X_test = X_train[subset_feature], X_test[subset_feature]

Ensuite, nous examinons les informations concernant le type de données de chaque colonne.

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1047 entries, 1118 to 684
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   embarked  1045 non-null   category
 1   sex       1047 non-null   category
 2   pclass    1047 non-null   int64   
 3   age       841 non-null    float64 
 4   fare      1046 non-null   float64 
dtypes: category(2), float64(2), int64(1)
memory usage: 35.0 KB


Nous pouvons voir que les colonnes `embarked` et `sex` ont été étiquetées comme des colonnes de catégories lors du chargement des données avec `fetch_openml`. Par conséquent, nous pouvons utiliser cette information pour envoyer les colonnes catégorielles au `categorical_transformer` transformateur catégoriel et les colonnes restantes au `numerical_transformer`.

**Note** - En pratique, vous devrez gérer vous-même le type de données de la colonne. Si vous voulez que certaines colonnes soient considérées comme des catégories, vous devrez les convertir en colonnes catégorielles. Si vous utilisez pandas, vous pouvez consulter leur documentation sur les [**données catégorielles**](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html).

In [9]:
from sklearn.compose import make_column_selector as selector

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)


clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))
clf

model score: 0.798


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001C2B0A07550>),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=50,
                                                                                    score_func=<function chi2 at 0x000001C2AF6E6440>))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001C2B0B10250>)])),
                ('classifier', LogisticRegression())])

Le score résultant n'est pas exactement le même que celui du pipeline précédent car le sélecteur basé sur le type dtype considère la colonne `pclass` comme une caractéristique numérique au lieu d'une caractéristique catégorielle comme précédemment :

In [10]:
selector(dtype_exclude="category")(X_train)

['pclass', 'age', 'fare']

In [11]:
selector(dtype_include="category")(X_train)

['embarked', 'sex']

## Utilisation du pipeline de prédiction dans une recherche de grille

La recherche de grille peut également être effectuée sur les différentes étapes de prétraitement définies dans l'objet `ColumnTransformer`, ainsi que sur les hyperparamètres du classificateur en tant que partie du `Pipeline`. Nous rechercherons à la fois la stratégie de complétion des données numériques et le paramètre de régularisation de la régression logistique en utilisant [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Cette recherche d'hyperparamètres sélectionne aléatoirement un nombre fixe de paramètres de configuration défini par `n_iter`. En alternative, on peut utiliser [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), mais le produit cartésien de l'espace de paramètres sera évalué.

In [12]:
param_grid = {
    "preprocessor__num__imputer__strategy": ["mean", "median"],
    "preprocessor__cat__selector__percentile": [10, 30, 50, 70],
    "classifier__C": [0.1, 1.0, 10, 100],
}

search_cv = RandomizedSearchCV(clf, param_grid, n_iter=10, random_state=0)
search_cv

RandomizedSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x000001C2B0A07550>),
                                                                              ('cat',
                                                                               Pipeline(steps=[('encoder',
                                                                                                OneHotEncoder(handle_unknown='ignore'))...
                                                                                                                 score_func=<function chi2 at 0x000001C2AF6E6440>))]),
                                                                               <sklearn.compose._column_transformer.make_column_selector object at 0x000001C2B0B10250>)])),
                                             ('classifier',
                                              LogisticRegression())]),
                   param_distributions={'classifier__C': [0.1, 1.0, 10, 100],
                                        'preprocessor__cat__selector__percentile': [10,
                                                                                    30,
                                                                                    50,
                                                                                    70],
                                        'preprocessor__num__imputer__strategy': ['mean',
                                                                                 'median']},
                   random_state=0)

L'appel à `fit` déclenche la recherche par validation croisée pour la meilleure combinaison d'hyperparamètres :

In [13]:
search_cv.fit(X_train, y_train)

print("Best params:")
print(search_cv.best_params_)

Best params:
{'preprocessor__num__imputer__strategy': 'mean', 'preprocessor__cat__selector__percentile': 30, 'classifier__C': 100}


Les scores de validation croisée internes obtenus par ces paramètres sont :

In [14]:
print(f"Internal CV score: {search_cv.best_score_:.3f}")

Internal CV score: 0.786


Nous pouvons également examiner les résultats de la recherche de grille les plus importants en tant que dataframe pandas :

In [15]:
import pandas as pd

cv_results = pd.DataFrame(search_cv.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[
    [
        "mean_test_score",
        "std_test_score",
        "param_preprocessor__num__imputer__strategy",
        "param_preprocessor__cat__selector__percentile",
        "param_classifier__C",
    ]
].head(5)

,mean_test_score,std_test_score,param_preprocessor__num__imputer__strategy,param_preprocessor__cat__selector__percentile,param_classifier__C
7,0.786015,0.031020,mean,30,100
0,0.785063,0.030498,median,30,1.0
2,0.785063,0.030498,mean,30,1.0
4,0.785063,0.030498,mean,10,10
3,0.783149,0.030462,mean,30,0.1


Les meilleurs hyperparamètres ont été utilisés pour ré-ajuster un modèle final sur l'ensemble complet d'entraînement. Nous pouvons évaluer ce modèle final sur des données de test retenues qui n'ont pas été utilisées pour l'ajustement des hyperparamètres.

In [16]:
print(
    "accuracy of the best model from randomized search: "
    f"{search_cv.score(X_test, y_test):.3f}"
)

accuracy of the best model from randomized search: 0.798
